In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd


In [ ]:
# Utilizamos dnn puesto que nuestro MCTS no lo creamos desde el inicio, sino que creamos un árbol en cada movimiento para ahorrar poder de cómputo
dataset = pd.read_csv("/content/drive/My Drive/Chess/preprocessed.csv", header=None)

In [ ]:
# piezas = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
# piezas = ['.', 'P', 'N', 'B', 'R', 'Q', 'K', 'p', 'n', 'b', 'r', 'q', 'k']
temp = dataset.to_numpy()
a = temp[1,:]
board = np.array([a[0:8],a[8:16],a[16:24],a[24:32],a[32:40],a[40:48],a[48:56],a[56:64]])
board

array([[ 0.,  0.,  6.,  0.,  0.,  0.,  0.,  0.],
       [11.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., 12.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [ ]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64
0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5200.0
1,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5300.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5700.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5400.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5700.0


In [ ]:
# el resultado del dataset los pasamos a la variable y 
y = dataset[64].values /10000

In [ ]:
# Todos los valores a excepción del resultado se lo pasamos a la X
X = dataset.drop(64, axis=1)

In [ ]:
X.shape

(164431, 64)

In [ ]:
# Se realiza el split necesario paras la creación de la dnn
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

#Creación de la dnn con capas Dense

def create_modelo():
  model = Sequential()

  model.add(Dense(64, input_dim=64))
  model.add(Dense(32))
  model.add(Dense(16)) #Comentar de ser necesario
  model.add(Dense(8))  #para alternar la estructura
  model.add(Dense(4))  #del modelo
  model.add(Dense(2)) 
  model.add(Dense(1))
  
  # Optimizador descenso gradiente
  model.compile(optimizer='adam',
              loss='mse', metrics=['accuracy'])

  model.summary()
  return model

In [ ]:
model = create_modelo()
history = model.fit(train_X, train_y, epochs=100, batch_size=32, validation_data=(test_X, test_y))


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_11 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_13 (Dense)             (None, 1)                

In [ ]:
model1 = create_modelo()
history1 = model1.fit(train_X, train_y, epochs=64, batch_size=32, validation_data=(test_X, test_y))

In [ ]:
model.evaluate(test_X, test_y)

1285/1285 [==============================] - 1s 1ms/step - loss: 0.0136 - accuracy: 0.0468


[0.01360059529542923, 0.046803541481494904]

In [ ]:
model.save("/content/drive/My Drive/Chess/model64_0_1.h5") #64 dividiendo en medios 
#100 epocas

In [ ]:
#model.save("/content/drive/My Drive/Chess/model3.h5")

In [ ]:
train_X = np.asarray(train_X)
